In [1]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import argparse
from tensorflow.python.ops import rnn_cell

import evaluation

PATH_TO_TRAIN = '/Users/qiangwang/Data/recsys/yoochoose-data/yoochoose-clicks.dat'
PATH_TO_TEST = '/Users/qiangwang/Data/recsys/yoochoose-data/yoochoose-test.dat'

In [2]:
is_training = False
layers = 1
rnn_size = 100
n_epochs = 3
batch_size = 50
dropout_p_hidden=1
learning_rate = 0.001
decay = 0.96
decay_steps = 1e4
sigma = 0
init_as_normal = False
reset_after_session = True
session_key = 'SessionId'
item_key = 'ItemId'
time_key = 'Timestamp'
grad_cap = 0
test_model = 2
checkpoint_dir = './checkpoint'
loss = 'cross-entropy'
final_act = 'softmax'
hidden_act = 'tanh'
n_items = -1

item_key = 'ItemId'

In [3]:
data = pd.read_csv(PATH_TO_TRAIN, header=None, names=['SessionId', 'Timestamp', 'ItemId', 'Label'], sep=',', dtype={'SessionId': np.int32, 'ItemId': np.int64}, parse_dates=['Timestamp'])
valid = pd.read_csv(PATH_TO_TEST, header=None, names=['SessionId', 'Timestamp', 'ItemId', 'Label'], sep=',', dtype={'SessionId': np.int32, 'ItemId': np.int64}, parse_dates=['Timestamp'])

/Users/qiangwang/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
itemids = data[item_key].unique()
n_items = len(itemids)
itemidmap = pd.Series(data=np.arange(n_items), index=itemids)
data = pd.merge(data, pd.DataFrame({item_key:itemids, 'ItemIdx':itemidmap[itemids].values}), on=item_key, how='inner')

In [5]:
data.sort_values(['SessionId', 'Timestamp'], inplace=True)
offset_sessions = np.zeros(data['SessionId'].nunique()+1, dtype=np.int32)
offset_sessions[1:] = data.groupby('SessionId').size().cumsum()

In [6]:
n_items

52739

In [7]:
X = tf.placeholder(tf.int32, [batch_size], name='input')
Y = tf.placeholder(tf.int32, [batch_size], name='output')
global_state = [tf.placeholder(tf.float32, [batch_size, rnn_size], name='rnn_state') for _ in range(layers)]
global_step = tf.Variable(0, name='global_step', trainable=False)

with tf.variable_scope('gru_layer'):
    sigma = sigma if sigma != 0 else np.sqrt(6.0 / (n_items + rnn_size))
    initializer = tf.random_normal_initializer(mean=0, stddev=sigma)
#     initializer = tf.random_uniform_initializer(minval=-sigma, maxval=sigma)
    embedding = tf.get_variable('embedding', [n_items, rnn_size], initializer=initializer)
    softmax_W = tf.get_variable('softmax_w', [n_items, rnn_size], initializer=initializer)
    softmax_b = tf.get_variable('softmax_b', [n_items], initializer=tf.constant_initializer(0.0))

    cell = rnn_cell.GRUCell(rnn_size, activation=tf.nn.tanh)
    drop_cell = rnn_cell.DropoutWrapper(cell, output_keep_prob=dropout_p_hidden)
    stacked_cell = rnn_cell.MultiRNNCell([drop_cell] * layers)
    
    inputs = tf.nn.embedding_lookup(embedding, X)
    output, state = stacked_cell(inputs, tuple(global_state))
#     init_state = cell.zero_state(batch_size, tf.float32)
#     rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, rnn_inputs, initial_state=init_state)
    
    final_state = state

'''
Use other examples of the minibatch as negative samples.
'''
sampled_W = tf.nn.embedding_lookup(softmax_W, Y)
sampled_b = tf.nn.embedding_lookup(softmax_b, Y)
logits = tf.matmul(output, sampled_W, transpose_b=True) + sampled_b
yhat = tf.nn.tanh(logits)
cost = tf.reduce_mean(-tf.log(tf.diag_part(yhat)+1e-24))

In [ ]:
# logits = tf.matmul(output, softmax_W, transpose_b=True) + softmax_b
# self.yhat = self.final_activation(logits)
# self.lr = tf.maximum(1e-5,tf.train.exponential_decay(self.learning_rate, self.global_step, self.decay_steps, self.decay, staircase=True)) 

# '''
# Try different optimizers.
# '''
# #optimizer = tf.train.AdagradOptimizer(self.lr)
# optimizer = tf.train.AdamOptimizer(self.lr)
# #optimizer = tf.train.AdadeltaOptimizer(self.lr)
# #optimizer = tf.train.RMSPropOptimizer(self.lr)

# tvars = tf.trainable_variables()
# gvs = optimizer.compute_gradients(self.cost, tvars)
# if self.grad_cap > 0:
#     capped_gvs = [(tf.clip_by_norm(grad, self.grad_cap), var) for grad, var in gvs]
# else:
#     capped_gvs = gvs 
# self.train_op = optimizer.apply_gradients(capped_gvs, global_step=self.global_step)

In [10]:
print('fitting model...')
# lr = tf.maximum(1e-5,tf.train.exponential_decay(learning_rate, global_step, decay_steps, decay, staircase=True)) 
# optimizer = tf.train.AdamOptimizer(lr)
# tvars = tf.trainable_variables()
# gvs = optimizer.compute_gradients(cost, tvars)
# train_op = optimizer.apply_gradients(gvs, global_step=global_step)
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for epoch in range(n_epochs):
    epoch_cost = []
    state = [np.zeros([batch_size, rnn_size], dtype=np.float32) for _ in range(layers)]
    session_idx_arr = np.arange(len(offset_sessions)-1)
    iters = np.arange(batch_size)
    maxiter = iters.max()
    start = offset_sessions[session_idx_arr[iters]]
    end = offset_sessions[session_idx_arr[iters]+1]
    finished = False
    while not finished:
        minlen = (end-start).min()
        out_idx = data.ItemIdx.values[start]
        for i in range(minlen-1):
            in_idx = out_idx
            out_idx = data.ItemIdx.values[start+i+1]
            # prepare inputs, targeted outputs and hidden states
            fetches = [cost, final_state, global_step, lr, train_op]
            feed_dict = {X: in_idx, Y: out_idx}
            for j in range(layers):
                feed_dict[global_state[j]] = state[j]
            
            cost, state, step, lr, _ = sess.run(fetches, feed_dict)
            epoch_cost.append(cost)
            if np.isnan(cost):
                print(str(epoch) + ':Nan error!')
                error_during_train = True
                break
            if step == 1 or step % decay_steps == 0:
                avgc = np.mean(epoch_cost)
                print('Epoch {}\tStep {}\tlr: {:.6f}\tloss: {:.6f}'.format(epoch, step, lr, avgc))
        start = start+minlen-1
        mask = np.arange(len(iters))[(end-start)<=1]
        for idx in mask:
            maxiter += 1
            if maxiter >= len(offset_sessions)-1:
                finished = True
                break
            iters[idx] = maxiter
            start[idx] = offset_sessions[session_idx_arr[maxiter]]
            end[idx] = offset_sessions[session_idx_arr[maxiter]+1]
        if len(mask) and reset_after_session:
            for i in range(layers):
                state[i][mask] = 0
    
    avgc = np.mean(epoch_cost)
    if np.isnan(avgc):
        print('Epoch {}: Nan error!'.format(epoch, avgc))
        error_during_train = True
    saver.save(sess, '{}/gru-model'.format(checkpoint_dir), global_step=epoch)

fitting model...
0:Nan error!


TypeError: Fetch argument nan has invalid type <class 'numpy.float32'>, must be a string or Tensor. (Can not convert a float32 into a Tensor or Operation.)

In [14]:
# data.groupby('SessionId').size()